In [33]:
## First Part: Load the data: 
import pandas as pd 
import numpy as np

def shuffle(x_,y_):
    s = np.arange(x_.shape[0])
    s = np.random.shuffle(s)

    x_re = x_[s]
    y_re = y_[s]

    x_re = np.reshape(x_re,(len(x_),48,48))
    y_re = np.reshape(y_re,(len(y_)))
    return x_re,y_re

def read_fer(path):
    # train_path = "C:\\Users\\Jiaming Nie\\Documents\\Work-DeepGlint\Facial\datasets\\train.csv"
    data = pd.read_csv(path, dtype='a')
    label = np.array(data['emotion'])
    img_data = np.array(data['pixels'])

    N_sample = label.size

    x_data = np.zeros((N_sample, 48 * 48))
    # train_label = np.zeros((N_sample, 7), dtype=int)
    y_label = np.zeros(N_sample, dtype=int)
    # print(train_label)

    for i in range(N_sample):
        x = img_data[i]
        x = np.fromstring(x, dtype=float, sep=' ')
        x_max = x.max()
        x = x / (x_max + 0.0001)
        # print x_max
        # print x
        x_data[i] = x
        y_label[i] = int(label[i])
        # train_label[i, label[i]] = 1 #This step seems direct one-hot encoding
        # print(y_label[i])
        #    img_x = np.reshape(x, (48, 48))
        #    plt.subplot(10,10,i+1)
        #    plt.axis('off')
        #    plt.imshow(img_x, plt.cm.gray)

    x_data = np.reshape(x_data,(len(x_data),48,48))
    return x_data, y_label

def ReadData_fer():
    # ubuntu path
    #path_train = "/home/jiaming/code/DeepGlint-Work/Facial/datasets/train.csv"
    #path_test = "/home/jiaming/code/DeepGlint-Work/Facial/datasets/test.csv"

    # windows path
    path_train = "/train/trainset/1/train.csv"
    path_test = "/train/trainset/1/test.csv"
    path_vali = "/train/trainset/1/val.csv"
    
    x_train, y_train = read_fer(path_train)
    x_test, y_test = read_fer(path_test)
    x_vali, y_vali = read_fer(path_vali)

    x_train, y_train = shuffle(x_train, y_train)
    x_test, y_test = shuffle(x_test, y_test)
    x_vali, y_vali = shuffle(x_vali, y_vali)

    return x_train,y_train,x_test,y_test,x_vali,y_vali

def zca_whitening(X):
    """
        Function to compute ZCA whitening matrix (aka Mahalanobis whitening).
        INPUT:  X: [M x N] matrix.
            Rows: Variables
            Columns: Observations
        OUTPUT: ZCAMatrix: [M x M] matrix
        """
    mean_ = np.mean(X)
    X = X - mean_
    # Covariance matrix [column-wise variables]: Sigma = (X-mu)' * (X-mu) / N
    sigma = np.cov(X, rowvar=True)  # [M x M]
    # Singular Value Decomposition. X = U * np.diag(S) * V
    U, S, V = np.linalg.svd(sigma)
    # U: [M x M] eigenvectors of sigma.
    # S: [M x 1] eigenvalues of sigma.
    # V: [M x M] transpose of U
    # Whitening constant: prevents division by zero
    epsilon = 0.1
    # ZCA Whitening matrix: U * Lambda * U'
    ZCAMatrix = np.dot(U, np.dot(np.diag(1.0 / np.sqrt(S + epsilon)), U.T))  # [M x M]
    return ZCAMatrix

def normalization(x_):

    length = len(x_)

    for i in range(length):
        x_[i] = zca_whitening(x_[i])

    return x_

def oneHot(y_):
    # Function to encode output labels from number indexes 
    # e.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    y_ = y_.reshape(len(y_))
    n_values = int(np.max(y_)) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

In [34]:
x_train,y_train,x_test,y_test,x_vali,y_vali = ReadData_fer()

# Normalization
# x_train = normalization(x_train)
# x_test = normalization(x_test)
# x_vali = normalization(x_vali)

x_train = x_train.reshape((len(x_train), 48, 48, 1))
x_test = x_test.reshape((len(x_test), 48, 48, 1))
x_vali = x_vali.reshape((len(x_vali),48,48,1))

print(x_train.shape)
print(x_test.shape)

(28709, 48, 48, 1)
(3589, 48, 48, 1)


In [35]:
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        title='Normalized confusion matrix'
    else:
        title='Confusion matrix'

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
## multiclass or binary report
## If binary (sigmoid output), set binary parameter to True
def full_multiclass_report(model,
                           x,
                           y_true,
                           classes,
                           batch_size=32,
                           binary=False):

    # 1. Transform one-hot encoded y_true into their class number
    if not binary:
        y_true = np.argmax(y_true,axis=1)
    
    # 2. Predict classes and stores in y_pred
    y_pred = model.predict_classes(x, batch_size=batch_size)
    
    # 3. Print accuracy score
    print("Accuracy : "+ str(accuracy_score(y_true,y_pred)))
    
    print("")
    
    # 4. Print classification report
    print("Classification Report")
    print(classification_report(y_true,y_pred,digits=5))    
    
    # 5. Plot confusion matrix
    cnf_matrix = confusion_matrix(y_true,y_pred)
    print(cnf_matrix)
    plot_confusion_matrix(cnf_matrix,classes=classes)

In [44]:
## Simple CNN with 3 layers definition:
from keras.models import Sequential
from keras.layers import Convolution2D,Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import *
from keras.layers import Dense
import keras
import time 
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.regularizers import l2 #activity_l2

gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
tf.keras.backend.set_session(sess)


batch_size=64
epochs=40
num_class = 7
num_output = 7

model = Sequential()
#model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu',#input_shape=(48,48,1)))
model.add(Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(48, 48, 1),  kernel_initializer= 'glorot_uniform'))

model.add(Conv2D(32, (3, 3), padding="same", activation="relu", kernel_initializer= 'glorot_uniform',kernel_regularizer=l2(1e-5)))
#model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
model.add(Conv2D(32, (3, 3), padding="same", activation="relu", kernel_initializer= 'glorot_uniform',kernel_regularizer=l2(1e-5)))          
#model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding="same", activation="relu", kernel_initializer= 'glorot_uniform',kernel_regularizer=l2(1e-5)))
#model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding="same", activation="relu", kernel_initializer= 'glorot_uniform',kernel_regularizer=l2(1e-5)))
#model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding="same", activation="relu", kernel_initializer= 'glorot_uniform',kernel_regularizer=l2(1e-5)))
#model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding="same", activation="relu", kernel_initializer= 'glorot_uniform',kernel_regularizer=l2(1e-5)))
#model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding="same", activation="relu", kernel_initializer= 'glorot_uniform',kernel_regularizer=l2(1e-5)))
#model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding="same", activation="relu", kernel_initializer= 'glorot_uniform',kernel_regularizer=l2(1e-5)))
#model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64, activation='relu', kernel_initializer= 'glorot_uniform',kernel_regularizer=l2(1e-5)))
model.add(Dense(64, activation='relu', kernel_initializer= 'glorot_uniform',kernel_regularizer=l2(1e-5)))
model.add(Dense(7, activation='softmax',kernel_initializer= 'glorot_uniform',kernel_regularizer=l2(1e-5)))



# Compiling the CNN
sgd= SGD(lr=0.01,momentum=0.99, decay=0.9999, nesterov=True)

#adam = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer = sgd , loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()
print("Non-Training")
score = model.evaluate(x_train, to_categorical(y_train), verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])


results = []
start_time = time.time()
history = model.fit(x_train, to_categorical(y_train), 
           batch_size=batch_size, 
           epochs=epochs, 
           verbose=1, 
           validation_data=(x_vali, to_categorical(y_vali)),shuffle = True)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_160 (Conv2D)          (None, 48, 48, 32)        320       
_________________________________________________________________
conv2d_161 (Conv2D)          (None, 48, 48, 32)        9248      
_________________________________________________________________
conv2d_162 (Conv2D)          (None, 48, 48, 32)        9248      
_________________________________________________________________
dropout_26 (Dropout)         (None, 48, 48, 32)        0         
_________________________________________________________________
max_pooling2d_70 (MaxPooling (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_163 (Conv2D)          (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_164 (Conv2D)          (None, 24, 24, 64)        36928     
__________

28709/28709 [==============================] - 349s 12ms/step - loss: 1.8168 - acc: 0.2513 - val_loss: 1.8242 - val_acc: 0.2494
Epoch 39/40
28709/28709 [==============================] - 350s 12ms/step - loss: 1.8167 - acc: 0.2513 - val_loss: 1.8242 - val_acc: 0.2494
Epoch 40/40
28709/28709 [==============================] - 348s 12ms/step - loss: 1.8166 - acc: 0.2513 - val_loss: 1.8242 - val_acc: 0.2494
